In [1]:
import pandas as pd
import numpy as np
import datetime

In [23]:
df0 = pd.read_excel("D:\Payout input Q4-22-23.xlsx")

In [24]:
df0.head()

,Payment Type,Office Code,Office Name,Product Code,Product Name,Imd Code,Imd Channel,Business Channel Code,Business Channel Name,Claim Class Code,...,TDS,DUMMY COL1,DUMMY COL2,expense_gl_code,payment_gl_code,bank_gl_code,multiple_rec_flag,irda_status,irda_status_reason,irda_claim_status
0,Bank transfer,10101,Jammu,10401,Digit Health Care Plus Policy,1112660,AGEN - INDIVIDUAL AGENT,AG,D01,KE,...,0,NaN,NaN,10200001,11300037,11100033,NaN,PAID AND CLOSED,PAID AND CLOSED,Y
1,Bank transfer,10101,Jammu,20101,Digit Private Car Package Policy,1000643,POSP - POINT OF SALE PERSON,AG,D01,KK,...,533,NaN,NaN,10200001,11300037,11100033,NaN,PAID AND CLOSED,PAID AND CLOSED,Y
2,Bank transfer,10101,Jammu,20101,Digit Private Car Package Policy,1000643,POSP - POINT OF SALE PERSON,AG,D01,KK,...,203,NaN,NaN,10200001,11300037,11100033,NaN,PAID AND CLOSED,PAID AND CLOSED,Y
3,Bank transfer,10101,Jammu,20101,Digit Private Car Package Policy,1001070,POSP - POINT OF SALE PERSON,AG,D01,KK,...,127,NaN,NaN,10200001,11300037,11100033,NaN,PAID AND CLOSED,PAID AND CLOSED,Y
4,Bank transfer,10101,Jammu,20101,Digit Private Car Package Policy,1001856,POSP - POINT OF SALE PERSON,AG,D01,KK,...,139,NaN,NaN,10200001,11300037,11100033,NaN,PAID AND CLOSED,PAID AND CLOSED,Y


In [27]:
df = df0.drop(['Office Code','Office Name','Imd Code','Imd Channel','Buss Partner Code','Debit Account Number','insured_person','policy_holder','veh_reg_no','Office Gstin','Gst Claim Location','Receiver Email','Transaction Reference No','Clerk Name','Payment Utr No','payout_block_reason','buss_partner_advance_flag','advance_payout_flag','ex_showroom_price','qs_cession_flag','covid_gross_amt','hev_gross_amt','loss_cause','class_code','leader_claim_no','leader_policy_no','group_type','COL1','COL2','IDENTIFIER','REFERENCE NO','BENEFICIARY NAME','ACCOUNT NO','ADDRESS1','ADDRESS2','ADDRESS3','ADDRESS4','PAYABLE LOCATION CODE','PRINT LOCATION CODE','PAYMENT DETAILS','EMAILID','DEBIT ACCOUNT NO','INV DETAILS','GROSS AMT','GST','TDS','DUMMY COL1','DUMMY COL2','expense_gl_code','payment_gl_code','bank_gl_code','multiple_rec_flag','irda_status_reason','irda_claim_status'], axis = 1)

In [28]:
df.head()

,Payment Type,Product Code,Product Name,Business Channel Code,Business Channel Name,Claim Class Code,Claim Class Name,accounting_class_code,Booking Account Type,Receiver Type,...,retained_shared,master_policy_no,loss_date,claim_status,closing_date,risk_exp_date,PAYMENT TYPE,DATE,BANK CODE,irda_status
0,Bank transfer,10401,Digit Health Care Plus Policy,AG,D01,KE,ALL HOSPITALIZATION COVER,OD,claim account,POLICY_HOLDER,...,1.0,NaN,2023-02-24 00:00:00,CLOSED,2023-03-17,2024-01-22,ACH,2023-03-17,SBIN0001189,PAID AND CLOSED
1,Bank transfer,20101,Digit Private Car Package Policy,AG,D01,KK,OWN DAMAGE,OD,claim account,DIGIT PREFERRED REPAIR SHOP,...,1.0,NaN,2023-02-22 18:29:00,CLOSED,2023-02-28,2023-08-22,ACH,2023-02-28,JAKA0PATTEL,PAID AND CLOSED
2,Bank transfer,20101,Digit Private Car Package Policy,AG,D01,KK,OWN DAMAGE,OD,claim account,DIGIT PREFERRED REPAIR SHOP,...,1.0,NaN,2023-03-12 18:30:00,CLOSED,2023-03-21,2023-08-22,ACH,2023-03-21,JAKA0PATTEL,PAID AND CLOSED
3,Bank transfer,20101,Digit Private Car Package Policy,AG,D01,KK,OWN DAMAGE,OD,claim account,DIGIT PREFERRED REPAIR SHOP,...,1.0,NaN,2023-02-02 13:00:00,CLOSED,2023-02-23,2023-03-17,ACH,2023-02-23,SBIN0001291,PAID AND CLOSED
4,Bank transfer,20101,Digit Private Car Package Policy,AG,D01,KK,OWN DAMAGE,OD,claim account,DIGIT PREFERRED REPAIR SHOP,...,1.0,NaN,2022-12-27 00:00:00,CLOSED,2023-01-05,2023-01-10,ACH,2023-01-05,MAHB0001240,PAID AND CLOSED


In [29]:
df['Policy Sum Insured'] = df['Policy Sum Insured'].fillna(0)
df['Gross Amt As per ABS'] = df['Gross Amt As per ABS'].fillna(0)

In [30]:
#Sum insured is 0
suminsured_is_0 = df.loc[((df['Policy Sum Insured'] == 0))]

In [31]:
# Claim amt > sum insured
df["claim>suminsured"] = df["Policy Sum Insured"] - df["Gross Amt As per ABS"]

In [32]:
Claim_more_than_suminsured=df.loc[((df['claim>suminsured'] <0) & (df['Status'] != 'Deleted'))]

In [33]:
#validation of pan and GST length
df["Receiver Pan_Length"] = df["Receiver Pan"].str.len()
df["Receiver GST_Length"] = df["Receiver Gst"].str.len()
df["IFSC_Length"] = df["Credit Ifsc"].str.len()

In [34]:
PAN_Length_not_10=df.loc[((df['Receiver Pan_Length'] !=10) & (df['Status'] != 'Deleted') & (df['Gross Amt As per ABS'] >100000))]

In [35]:
GST_Length_not_15=df.loc[((df['Receiver GST_Length'] !=15) & (df['Status'] != 'Deleted'))]

In [36]:
IFSC_lessthan_11=df.loc[((df['IFSC_Length'] !=11) & (df['Status'] == 'Paid'))]

In [37]:
#Gross amt is 0 and TDS is deducted
Grsamt0_TDSdeducted=df.loc[((df['Gross Amt As per ABS'] <=0) & (df['Status'] != 'Deleted') & (df['Tds Amt'] >0))]

In [38]:
#Gross amt is 0 and GST is Positive
Grsamt0_GSTpositive=df.loc[((df['Gross Amt As per ABS'] <=0) & (df['Status'] != 'Deleted') & (df['Gst Amt'] >0))]

In [39]:
#Risk expiry should not be before risk inception
#Claim booking date should be after risk inception date
df["Risk_exp_date<Risk_inc_date"] = (df["risk_exp_date"]-df["risk_inc_date"])/np.timedelta64(1,'D')
df["Booking_date<Risk_inc_date"] = (df["Booking Date"]-df["risk_inc_date"])/np.timedelta64(1,'D')

In [40]:
Risk_exp_date_before_Risk_inc_date=df.loc[((df['Risk_exp_date<Risk_inc_date'] <0) & (df['Status'] != 'Deleted'))]
Booking_date_before_Risk_inc_date=df.loc[((df['Booking_date<Risk_inc_date'] <0) & (df['Status'] != 'Deleted'))]
#booking date after payment date
#Loss date after booking date
#invoice date after payment date
#booking date after loss date

In [41]:
#To draw pivot and verift all line items in payreport to finance logic
df["Gross amt as per finance vs ABS"] = df["Gross Amt As per ABS"] - df["Gross Amount as per Finance Logic"]
financelogicvsabs=df.loc[((df['Gross amt as per finance vs ABS'] !=0) & (df['Status'] == 'paid'))]

In [42]:
#Net claim recomputation
df["Netclaim_recomputation"] = df["Gross Amt As per ABS"] + df["Deduction Amt"] + df["Gst Amt"] + df["Advance Amt"] + df["Tds Amt"]

In [43]:
df["Netclaim_check"] = df["NET AMOUNT"] - df["Netclaim_recomputation"]

In [44]:
netclaimvalidation =df.loc[(df['Status'] != 'Deleted')]

In [45]:
#recovered claims having positive amt
recoverclaims=df.loc[((df['NET AMOUNT'] >0) & (df['Status'] == 'Recovery'))]

In [46]:
excel_file = pd.ExcelWriter("D:\Claims exception report_Q4_FY22-23.xlsx", engine='xlsxwriter')
suminsured_is_0.to_excel(excel_file, sheet_name='A.1')
Claim_more_than_suminsured.to_excel(excel_file, sheet_name='A.2')
PAN_Length_not_10.to_excel(excel_file, sheet_name='A.3')
GST_Length_not_15.to_excel(excel_file, sheet_name='A.4')
IFSC_lessthan_11.to_excel(excel_file, sheet_name='A.5')
Grsamt0_TDSdeducted.to_excel(excel_file, sheet_name='A.6')
Grsamt0_GSTpositive.to_excel(excel_file, sheet_name='A.7')
Risk_exp_date_before_Risk_inc_date.to_excel(excel_file, sheet_name='A.8')
Booking_date_before_Risk_inc_date.to_excel(excel_file, sheet_name='A.9')
financelogicvsabs.to_excel(excel_file, sheet_name='A.10')
netclaimvalidation.to_excel(excel_file, sheet_name='A.11')
recoverclaims.to_excel(excel_file, sheet_name='A.12')
excel_file.save()